In [1]:
import json
import pickle
import requests
import collections
import numpy as np
from decimal import Decimal

simple test

In [2]:
requests.post('http://127.0.0.1:8000/api/karl/reset')
payload_schedule = [
    {
        'text': 'which moved the capital to Harrisburg from Washington-on-the-Brazos.',
        'user_id': '2',
        'question_id': '17162',
        # 'user_accuracy': 0.0,
        # 'user_buzzratio': None,
        # 'user_count': 0.0,
        # 'question_accuracy': 0.0,
        # 'question_buzzratio': None,
        # 'question_count': 0.0,
        # 'times_seen': 0.0,
        # 'times_correct': 0.0,
        # 'times_wrong': 0.0,
        'label': None
    },
]
r = requests.post('http://127.0.0.1:8000/api/karl/schedule',
                  data=json.dumps(payload_schedule))
r = json.loads(r.text)
print(r)

{'probs': [[0.6215803623199463, 0.3784196376800537]], 'all_labels': [['correct', 'wrong']], 'card_order': [0]}


In [51]:
def generate_schedule(flashcards, study_flashcards, max_studies=20):
    # reset model
    requests.post('http://127.0.0.1:8000/api/karl/reset')
    
    # set hyperparameteers
    TARGET = 1.0
    params = {
        'learning_rate': 1e-3,
        'num_epochs': 40,
        'target': TARGET
    }
    r = requests.post('http://127.0.0.1:8000/api/karl/set_hyperparameter',
                      data=json.dumps(params))
    
    update_flashcards = []  # for user embedding update
    prev_rank_dict = dict() # qid -> previous ranking
    prev_prob_dict = dict() # qid -> previous probability
    for study_iter in range(max_studies):
        if len(flashcards) <= 0:
            break
        # get new ranking
        r = requests.post('http://127.0.0.1:8000/api/karl/schedule',
                          data=json.dumps(flashcards))
        r = json.loads(r.text)
        probs = [x[0] for x in r['probs']] # probability of answering correctly
        card_order = np.argsort(np.abs(TARGET - np.asarray(probs))).tolist()
        # qid -> probability
        curr_prob_dict = {c['question_id']: p for c, p in zip(flashcards, probs)}
        # qid -> ranking
        curr_rank_dict = {c['question_id']: card_order.index(i) for i, c in enumerate(flashcards)}
        # qid -> index
        curr_index_dict = {c['question_id']: i for i, c in enumerate(flashcards)}
        
        # get ranking changes
        rank_diff = {}
        prob_diff = {}
        for card in flashcards:
            qid = card['question_id']
            if qid in prev_rank_dict:
                # +1 to account for removal of the first card
                rank_diff[qid] = curr_rank_dict[qid] - prev_rank_dict[qid]#  + 1
            if qid in prev_prob_dict:
                prob_diff[qid] = curr_prob_dict[qid] - prev_prob_dict[qid]
        prev_prob_dict = curr_prob_dict.copy()
        prev_rank_dict = curr_rank_dict.copy()
        
        def convert_rdiff(rdiff):
            if rdiff == 0:
                return '  '
            if rdiff > 0:
                return '⇩' + str(rdiff)
            if rdiff < 0: 
                return '⇧' + str(-rdiff)
        
        def convert_pdiff(pdiff):
            if abs(pdiff) < 0.001:
                return '  '
            if pdiff > 0.001:
                return '+' + str(round(pdiff, 3))
            if pdiff < -0.001:
                return str(round(pdiff, 3))
            
        # print top 10 cards
        print('------ cards sorted by distance to %s' % TARGET)
        for rank, ind in enumerate(card_order[:10]):
            card = flashcards[ind]
            qid = card['question_id']
            answer = card['answer']
            assert curr_rank_dict[qid] == rank
            print('\t'.join([
                str(rank),
                convert_rdiff(rank_diff.get(qid, 0)),
                str(round(curr_prob_dict[qid], 3)),
                convert_pdiff(prob_diff.get(qid, 0)),
                answer
            ]))
        
        def print_diff(qid, answer):
            print('\t'.join([
                str(curr_rank_dict.get(qid, 0)),
                convert_rdiff(rank_diff.get(qid, 0)),
                convert_pdiff(prob_diff.get(qid, 0)),
                answer
            ]))
            
        print('------ largest increase in ranking')
        for qid, rdiff in rank_diff_sorted[:-5:-1]:
            if rdiff >= 0:
                break
            answer = flashcards[curr_index_dict[qid]]['answer']
            print_diff(qid, answer)
        print('------ largest increase in probability')
        prob_diff_sorted = sorted(prob_diff.items(), key=lambda x: -x[1])
        for qid, pdiff in prob_diff_sorted[:5]:
            if pdiff <= 0.001:
                break
            answer = flashcards[curr_index_dict[qid]]['answer']
            print_diff(qid, answer)
        print('------ largest decrease in ranking')
        rank_diff_sorted = sorted(rank_diff.items(), key=lambda x: -x[1])
        for qid, rdiff in rank_diff_sorted[:5]:
            if rdiff <= 0:
                break
            answer = flashcards[curr_index_dict[qid]]['answer']
            print_diff(qid, answer)
        print('------ largest decrease in probability')
        for qid, pdiff in prob_diff_sorted[:-5:-1]:
            if pdiff >= -0.001:
                break
            answer = flashcards[curr_index_dict[qid]]['answer']
            print_diff(qid, answer)
         
        # update card pool
        # update_flashcards = [study_flashcards[study_iter]]
        show_card = flashcards[card_order[0]]
        update_flashcards = [show_card]
        if show_card['label'] == 'correct' or show_card['label'] == 'wrong':
            flashcards.pop(card_order[0])
            for qid, rank in prev_rank_dict.items():
                if qid != show_card['question_id']:
                    prev_rank_dict[qid] = rank - 1
        
        # update model
        r = requests.post('http://127.0.0.1:8000/api/karl/update',
                          data=json.dumps(update_flashcards))
        r = json.loads(r.text)
        # print(r['loss'])
        
        print()
        print('------', show_card['label'], show_card['answer'], '-------')
        print()

In [ ]:
# protobowl data
with open('data/withanswer.question.json') as f:
    proto_questions = json.load(f)
proto_questions = {q['qid']: q for q in proto_questions}

with open('data/protobowl_byuser_dict_alldata.json') as f:
    protobowl_byuser_dict_alldata = json.load(f)

In [ ]:
def test_protobowl(max_cards=20, max_studies=20):
    # generate protobowl flashcards
    uid_1 = list(protobowl_byuser_dict_alldata['data'].keys())[1]
    print('uid', uid_1)
    
    # create set of flashcards from study record
    flashcards = []
    user_questions = protobowl_byuser_dict_alldata['data'][uid_1]['questions_per_user']
    user_labels = protobowl_byuser_dict_alldata['data'][uid_1]['accuracy_per_user']
    qid_set = set() # for dedup
    for qid, label in zip(user_questions[:max_cards], user_labels[:max_cards]):
        # entries are ranked by date
        if qid in qid_set:
            # question dedup, only add the first occurrence
            continue
        qid_set.add(qid)
        flashcards.append({
                'text': proto_questions[qid]['text'],
                'user_id': uid_1,
                'question_id': qid,
                'label': 'correct' if label else 'wrong',
                'answer': proto_questions[qid]['answer']
            })
    print('# flashcards', len(flashcards))
    print()
    
    generate_schedule(flashcards, proto_questions, max_studies)

In [4]:
# load jeopardy data
jeopardy_questions_df = pickle.load(open('data/jeopardy_358974_questions_20190612.pkl', 'rb'))
jeopardy_records_df = pickle.load(open('data/jeopardy_310326_question_player_pairs_20190612.pkl', 'rb'))

jeopardy_questions_dict = jeopardy_questions_df.to_dict()
jeopardy_questions = {}
for i in jeopardy_questions_dict['clue'].keys():
    jeopardy_questions[jeopardy_questions_dict['questionid'][i]] = {
        'text': jeopardy_questions_dict['clue'][i],
        'answer': str(jeopardy_questions_dict['answer'][i]) + ' (%s)' % jeopardy_questions_dict['category'][i],
    }

In [26]:
def test_jeopardy(max_cards=20, max_studies=20):
    # generate jeopardy flashcard
    # uid_1 = 'player_019'
    # print('uid', uid_1)
    # jeopardy_set = jeopardy_records_df.groupby('player_id').get_group(uid_1)
    
    category_mapping = {}
    for row in jeopardy_questions_df.itertuples():
        category_mapping[row.questionid] = row.category
    jeopardy_records_df['category'] = jeopardy_records_df['question_id'].apply(lambda x: category_mapping.get(x, None))
    
    # most popular categories
    category_counter = collections.Counter(jeopardy_questions_df.category.tolist())
    top_five_categories = [x[0] for x in category_counter.most_common(10)]
    top_five_categories = [x for x in top_five_categories if x != 'POTPOURRI'][:5]
    print(top_five_categories)
    
    # most answered questions within each category
    N_QUESTIONS = 40
    selected_qids = []
    for category in top_five_categories:
        df = jeopardy_records_df[jeopardy_records_df.category == category]
        question_counter = collections.Counter(df.question_id)
        selected_qids += [x[0] for x in question_counter.most_common(N_QUESTIONS)]
    # print(selected_qids)
    
    # filter records and get fake label
    jeopardy_records_filtered_df = jeopardy_records_df[jeopardy_records_df.question_id.isin(selected_qids)]
    jeopardy_set_df = jeopardy_records_filtered_df[['question_id', 'correct']]
    jeopardy_set_df['correct'] = jeopardy_set_df['correct'].apply(lambda x: int(x))
    jeopardy_set_df = jeopardy_set_df.groupby('question_id').mean()
    jeopardy_set_df['correct'] = jeopardy_set_df['correct'].apply(lambda x: 1 if x > 0 else -1)
    
    # create set of flashcards from study record
    flashcards = []
    for row in jeopardy_set_df.itertuples():
        if len(flashcards) >= max_cards:
            break
        qid = row.Index
        flashcards.append({
                'text': jeopardy_questions[qid]['text'],
                'user_id': 'diagnosis',
                'question_id': qid,
                'label': 'correct' if row.correct == 1 else 'wrong',
                'answer': jeopardy_questions[qid]['answer']
            })
    print('# flashcards', len(flashcards))
    print()
    study_flashcards = flashcards
    
    generate_schedule(flashcards, study_flashcards, max_studies=20)

In [52]:
test_jeopardy(max_cards=1000)

['SCIENCE', 'LITERATURE', 'AMERICAN HISTORY', 'SPORTS', 'HISTORY']
# flashcards 200



/fs/clip-quiz/shifeng/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


------ cards sorted by distance to 1.0
0	  	0.773	  	Mogadishu (HISTORY)
1	  	0.772	  	The Continental Congress (AMERICAN HISTORY)
2	  	0.765	  	Radical Republicans (AMERICAN HISTORY)
3	  	0.763	  	atmosphere (SCIENCE)
4	  	0.763	  	eagle (SPORTS)
5	  	0.757	  	(Henry) Comstock (AMERICAN HISTORY)
6	  	0.753	  	Stanford (SPORTS)
7	  	0.752	  	steam engines (AMERICAN HISTORY)
8	  	0.751	  	L.A. Kings (SPORTS)
9	  	0.749	  	Annika Sorenstam (SPORTS)
------ largest decrease in ranking
------ largest increase in ranking
------ largest increase in probability
------ largest decrease in probability

------ correct Mogadishu (HISTORY) -------

------ cards sorted by distance to 1.0
0	⇧12	0.903	+0.159	Louisiana Purchase (AMERICAN HISTORY)
1	⇧3	0.899	+0.142	(Henry) Comstock (AMERICAN HISTORY)
2	⇧77	0.896	+0.173	Robert Fulton (AMERICAN HISTORY)
3	⇩3	0.894	+0.122	The Continental Congress (AMERICAN HISTORY)
4	⇩3	0.892	+0.127	Radical Republicans (AMERICAN HISTORY)
5	⇧24	0.882	+0.149	Saratoga (SPORTS


------ correct Toronto Maple Leafs (SPORTS) -------

------ cards sorted by distance to 1.0
0	⇧1	0.999	+0.006	Annika Sorenstam (SPORTS)
1	⇧7	0.999	+0.01	one (SPORTS)
2	  	0.999	+0.009	Stanford (SPORTS)
3	⇧6	0.999	+0.01	Free Throw (SPORTS)
4	⇧9	0.998	+0.011	West Virginia (paid debt to Virginia) (AMERICAN HISTORY)
5	⇩2	0.998	+0.009	Paul Hamm (SPORTS)
6	⇧15	0.998	+0.012	eagle (SPORTS)
7	⇧18	0.998	+0.012	L. Frank Baum (LITERATURE)
8	⇧11	0.998	+0.011	steam engines (AMERICAN HISTORY)
9	⇩5	0.998	+0.009	Wife of Bath (LITERATURE)
------ largest decrease in ranking
118	⇩76	+0.003	Radical Republicans (AMERICAN HISTORY)
72	⇩67	+0.003	the League of Nations (HISTORY)
75	⇩65	+0.003	(Henry) Comstock (AMERICAN HISTORY)
119	⇩62	+0.006	mole (SCIENCE)
135	⇩55	+0.002	The Continental Congress (AMERICAN HISTORY)
------ largest increase in ranking
67	⇧57	+0.026	depression (HISTORY)
115	⇧47	+0.034	(Rutherford B.) Hayes (AMERICAN HISTORY)
57	⇧45	+0.024	hafnium (SCIENCE)
107	⇧45	+0.031	_A Tale Of Two Cities_ (L

KeyboardInterrupt: 